In [ ]:
## MOUNT GOOGLE DRIVE

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
## PACKAGE IMPORTS

import pandas as pd
import os
import numpy as np

In [ ]:
# REMOVE NO DATA ROWS
def remove_empty_rows(df):
    df = df.replace("", np.nan)
    return df.dropna(axis='rows')

In [ ]:
## IMPORT DATA FILES

THT_df = pd.read_csv('/content/gdrive/My Drive/ECE 225A Project (Walkability)/data/THT_Data_508_cleaned.csv') # transportation
health_df = pd.read_csv('/content/gdrive/My Drive/ECE 225A Project (Walkability)/data/combined_health.csv')
health_df = health_df[health_df.columns[1:]]
print(health_df)

           citystate  Binge_data    MH_Data  obesity_data
0        Abilene, TX   19.221234  14.602563     33.706916
1          Akron, OH   16.696070  16.903868     37.752952
2        Alameda, CA   18.446460   9.983403     20.024635
3         Albany, GA   15.509967  15.632831     34.740506
4    Albuquerque, NM   16.017994  14.294677     27.057895
..               ...         ...        ...           ...
459      Wyoming, MI   20.318978  15.278053     32.817754
460       Yakima, WA   14.438900  16.130709     37.222501
461      Yonkers, NY   16.776670  13.511176     27.300826
462   Youngstown, OH   13.192982  18.556215     45.503089
463         Yuma, AZ   17.855079  13.673813     36.707264

[464 rows x 4 columns]


In [ ]:
city = 'San Diego|Carlsbad'

# if (THT_df['Metropolitan Statistical Area'].str.contains(city)).any():
#     print(THT_df[THT_df['Metropolitan Statistical Area'].str.contains(city)])
    # print('{city} is in THT')

if (health_df['citystate'].str.contains(city)).any():
    print(health_df[health_df['citystate'].str.contains(city)])
    # print('{city} is in health_df')

         citystate  Binge_data    MH_Data  obesity_data
61    Carlsbad, CA   21.445219   9.753051     20.245101
362  San Diego, CA   20.052257  11.540465     22.102706


In [ ]:
# print hyphenated MSAs from THT

# new list to store combined health data
combined_health_msas = []

# find all combined MSAs
hyphen = THT_df[THT_df['Metropolitan Statistical Area'].str.contains('-')].reset_index()
combined_msas = list(hyphen['Metropolitan Statistical Area'])

for msa in combined_msas:
    # print('--------------------------------')
    # print(f'{len(health_df)}')

    citystate = msa.split(',')
    cities = citystate[0].strip().replace('-', '|')
    cities = cities.replace('||', '|')
    states = citystate[1].strip().replace('-', '|')
    states = states.replace('||', '|')

    health_subset = health_df[health_df['citystate'].str.contains(cities) & health_df['citystate'].str.contains(states)]
    new_binge = np.average(health_subset['Binge_data'])
    new_MH = np.average(health_subset['MH_Data'])
    new_obesity = np.average(health_subset['obesity_data'])

    # make new row with combined name
    new_row = [msa, new_binge, new_MH, new_obesity]

    # append to combined health_msas list
    combined_health_msas.append(new_row)

    # remove single towns from health_df
    # health_df = health_df[~health_df.isin(health_subset)].dropna(how = 'all')
    health_df = pd.merge(health_df, health_subset, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)




# make df from accumulated combined data
combined_df = pd.DataFrame(combined_health_msas, columns = health_df.columns)

# add combined_Df back into master health_df
health_df = remove_empty_rows(health_df.append(combined_df, ignore_index=True))

# raname citystate to Metropolitan Statistical Area
health_df = health_df.rename(columns = {'citystate': 'Metropolitan Statistical Area'})
# print(health_df)

#combine health_df with Master THT
final_df = THT_df.set_index('Metropolitan Statistical Area').combine_first(health_df.set_index('Metropolitan Statistical Area'))

# remove rows with missing data
final_df = remove_empty_rows(final_df)

# save as csv
final_df.to_csv('/content/gdrive/My Drive/ECE 225A Project (Walkability)/data/FINAL_DATA.csv')


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:518